<a href="https://colab.research.google.com/github/chaitu0032-debug/Hello_world/blob/main/some_random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow_datasets as tfds
mnist_data = tfds.load("plant_village")
mnist_train = mnist_data["train"]


In [3]:
images = []
labels = []
i = 0
for example in tfds.as_numpy(mnist_train):
  image, label = example['image'], example['label']
  images.append(image)
  labels.append(label)
  i = i +1
  if(i == 15000):
      break
len(mnist_train)

54303

In [4]:
len(images)

15000

In [5]:
import numpy as np

In [6]:
images = np.array(images)

In [7]:
labels = np.array(labels)

In [8]:
images.shape

(15000, 256, 256, 3)

In [9]:
labels.shape

(15000,)

In [28]:
from keras.utils import to_categorical
encoded = to_categorical(labels)
encoded.shape

labels = encoded

In [29]:
labels.shape

(15000, 38)

In [30]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input

input_tensor = Input(shape=(256, 256, 3))

model = ResNet50(input_tensor=input_tensor, weights='imagenet', include_top=False)

In [31]:
base_model = model

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(images,labels, test_size=0.25, random_state=1)


In [33]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [41]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(38, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
model.summary()
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model on the new data for a few epochs
model.fit(x=X_train,y =y_train,epochs = 7)

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [52]:
y_pred = model.predict(X_val)
y_pred.shape

(3750, 38)

In [53]:
y_pred = np.argmax(y_pred,axis = 1)

In [58]:
y_pred.shape
y_val = np.argmax(y_val,axis = 1)

In [59]:
z = y_pred


In [60]:
z

array([24, 16,  9, ..., 32, 32, 27])

In [61]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_val, y_pred)


array([[ 33,   0,   0, ...,   0,   0,   0],
       [  1,  52,   0, ...,   0,   0,   0],
       [  1,   0,  13, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ..., 104,   0,   0],
       [  0,   0,   0, ...,   0,  28,   0],
       [  0,   0,   0, ...,   0,   0, 352]])

In [62]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val,y_pred)

0.952

In [63]:
model.save("Resnet_finetuning")

INFO:tensorflow:Assets written to: Resnet_finetuning/assets


INFO:tensorflow:Assets written to: Resnet_finetuning/assets


In [64]:
model.save('path/to/location')

INFO:tensorflow:Assets written to: path/to/location/assets


INFO:tensorflow:Assets written to: path/to/location/assets


In [ ]:
from google.colab import drive
drive.mount('/content/drive')